<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/denoising/blob/main/figs/averaging_denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/vicente-gonzalez-ruiz/denoising/blob/main/figs/averae_denoising.ipynb)

# Averaging noisy instances of barb

In [ ]:
import math
from collections import namedtuple

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker

try:
    import cv2
except:
    !pip install cv2
    import cv2
    
try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io

try:
    import information_theory as IT
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    import information_theory as IT

import utils

# apt install cm-super-minimal
# apt install dvipng
plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})
#rc('text', usetex=True)
#rc('font', family='serif')
#plt.rc('text.latex', preamble=r'\usepackage{amsmath} `...')
#mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath} \usepackage{amsfonts}'

### barb

In [ ]:
Args = namedtuple("args", "input")
args = Args("http://www.hpca.ual.es/~vruiz/images/barb.png")

In [ ]:
image = skimage_io.imread(args.input)  # Ground Truth
image.mean()

In [ ]:
image.dtype

In [ ]:
#image = 255 * ((image - np.min(image)) / (np.max(image) - np.min(image)))

In [ ]:
plt.title(r"$\mathbf{X}=\mathrm{Barb}$")
plt.imshow(image, cmap="gray")
#plt.show()
#plt.savefig('barb.pdf', bbox_inches='tight')

In [ ]:
min_PSNR = 80  # Minimum quality to stop averaging

### barb_averaging

In [ ]:
gamma = 0.15 # 0.15
std_dev = 40 # 30

#Y = MPG_noise(image, std_dev, gamma, poisson_ratio=0.5).reshape(image.shape)
#Y = np.random.poisson(image.astype(np.float32) * _lambda) / _lambda
Y = utils.generate_MPGN(image, std_dev, gamma)

string  = r'$\hat{\mathbf{X}}=['
string += r"0.5(\mathbf{X} + \mathbf{N}_{\mathcal{N}"
string += r"(\sigma"
string += f"={std_dev})"
string += r'}'
string += r") + 0.5\mathbf{N}_{\mathcal{P}"
string += r"(\lambda"
string += f"={gamma}"
string += r'\cdot\mathbf{X})}'
string += f"/{gamma}"
string += ']'

string += f",~{IT.distortion.PSNR(Y.astype(np.uint8), image):.2f}"
string += r"~\text{dB}$"

plt.title(string)
print("min(image)=", np.min(image))
print("max(image)=", np.max(image))
#+ rf"$_(\sigma^2={std_dev})$") #" #+ \mathbf{N}_{{\mathcal P}(\lambda=\gamma\cdot\mathrm{Barb})}/\gamma$") 
print("aqui", np.max(Y), np.min(Y))
#Y = np.clip(a = Y, a_min=0, a_max=255)
plt.imshow(np.clip(a = Y, a_min=0, a_max=255), cmap="gray")
plt.savefig('0MMPG_barb.pdf', bbox_inches='tight')

In [ ]:
X = image
curves = []
for std_dev in range(0, 100, 20):
    gamma = 0.15 # std_dev / 100
    acc_denoised = np.zeros_like(X, dtype=np.float64)
    PSNR = 1
    iters = 1
    curve = []
    while PSNR < min_PSNR:
        #Y = np.random.poisson((X.astype(np.float32) + 1) * _lambda)
        #Y = Y - _lambda
        Y = utils.generate_MPGN(X, std_dev, gamma)
        #Y = Y - _lambda
        #Y = Y / _lambda
        print('a', np.max(Y), np.min(Y), np.mean(Y))
        Y = np.clip(a = Y, a_min=0, a_max=255)
        Y = Y.astype(np.uint8)
        #print('b', np.max(Y), np.min(Y))
        acc_denoised += Y
        denoised = acc_denoised/iters
        PSNR = IT.distortion.PSNR(denoised.astype(np.uint8), X)
        print(iters, PSNR, np.max(denoised), np.min(denoised))
        curve.append(PSNR)
        iters += 1
        #print('b', np.mean(denoised))
        #plt.imshow(denoised, cmap="gray")
        #plt.show()
        #input()

    curves.append(curve)
    print(iters)

In [ ]:
#plt.title(r"$\mathbb{A}_{\text{" + str(iters) + r"}}(\mathbf{N}" + rf"/{_lambda}" + r"), \mathbf{N}\sim\mathrm{Poisson}" + rf"(\lambda={_lambda}" + r"\cdot\mathrm{Barb})$")
string  = r"$\tilde{\mathbf{X}}"
string += r"^{[" + str(iters) + r"]}"
string += r"=\overline{\{\hat{\mathbf{X}}^{(i)}\}_{i=1}^{"
string += str(iters)
string += r"}},"
string += f"{IT.distortion.PSNR(denoised.astype(np.uint8), X):.2f}"
string += r"~\text{dB}$"
plt.title(string)
plt.imshow(np.clip(a=denoised, a_min=0, a_max=255), cmap="gray")
plt.savefig('barb_averaging.pdf', bbox_inches='tight')